##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Знакомство с тюнером Keras(Keras Tuner)

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/keras_tuner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

## Введение

Keras Tuner - это библиотека, которая помогает вам выбрать оптимальный набор гиперпараметров для вашей программы TensorFlow. Процесс выбора правильного набора гиперпараметров для вашего приложения машинного обучения (ML) называется *настройкой гиперпараметров* или *гипертонастройкой*.

Гиперпараметры - это переменные, которые управляют процессом обучения и топологией модели машинного обучения. Эти переменные остаются постоянными в процессе обучения и напрямую влияют на эффективность вашей модели. Гиперпараметры бывают двух типов:
1. **Гиперпараметры модели**, которые влияют на установки модели, такие как количество и глубина скрытых слоев.
2. **Гиперпараметры алгоритма**, которые влияют на скорость и качество алгоритма обучения, такие как скорость обучения для стохастического градиентного спуска (SGD) или количество ближайших соседей для классификатора k Nearest Neighbors(KNN).

В этом руководстве вы будете использовать Keras Tuner для выполнения гипертонастройки модели классификации изображений.

## Установка

In [ ]:
import tensorflow as tf
from tensorflow import keras

import IPython

Установка и импорт Keras Tuner.

In [ ]:
!pip install -U keras-tuner
import kerastuner as kt

## Загрузка и подготовка датасета

В этом руководстве вы будете использовать Keras Tuner, чтобы найти лучшие гиперпараметры для модели машинного обучения, которая классифицирует изображения одежды из [набора данных Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist). 

Загружаем данные.

In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [ ]:
# Нормализуем пиксели в значения между 0 и 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Определение модели

При построении модели для гипертонастройки вы также определяете пространство поиска гиперпараметров в дополнение к архитектуре модели. Модель, которую вы настроили для гипертюнинга, называется *гипермоделью *.

Вы можете определить гипермодель двумя способами:

* Используя функцию построителя модели
* Создав подкласс класса HyperModel из Keras Tuner API

Вы можете использовать два предопределенных класса HyperModel - [HyperXception](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperxception-class) и [HyperResNet](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperresnet-class) для приложений компьютерного зрения.

В этом руководстве вы будете использовать функцию построителя моделей для определения модели классификации изображений. Функция построителя моделей возвращает скомпилированную модель и использует гиперпараметры, которые вы определяете во время вызова функции.

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  
  # Настраиваем кол-во юнитов в первом слое Dense
  # Выбираем оптимальное значение между 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Настраиваем learning rate для оптимайзера
  # Выбираем оптимальное значение из 0.01, 0.001, или 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
  
  return model

## Создание экземпляра тюнера и гипертюнинг

Создайте экземпляр тюнера для выполнения гипертонастройки. Тюнер Keras имеет четыре доступных тюнера - RandomSearch, Hyperband, BayesianOptimization и Sklearn. В этом руководстве вы будете использовать тюнер [Hyperband](https://arxiv.org/pdf/1603.06560.pdf).

Чтобы создать экземпляр тюнера Hyperband, вы должны указать гипермодель, `objective`(метрику, на которую будет ориентироваться модель) для оптимизации и максимальное количество эпох для обучения (`max_epochs`).

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')                       

Алгоритм настройки Hyperband использует адаптивное распределение ресурсов и раннюю остановку, чтобы быстро перейти к высокопроизводительной модели. Это делается по подобию сетки спортивного чемпионата. Алгоритм обучает большое количество моделей в течение нескольких эпох и переносит на следующий этап только половину наиболее эффективных моделей. Hyperband определяет количество моделей для обучения в группе, вычисляя 1 + log<sub>`factor`</sub> (`max_epochs`) и округляя его до ближайшего целого числа.

Перед запуском поиска по гиперпараметрам определите ф-цию обратного вызова для очистки результатов обучения в конце каждого шага обучения.

In [ ]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

Запустите поиск гиперпараметров. Аргументы для метода поиска те же, что и для `tf.keras.model.fit` плюс колбек, определенный выше.

In [ ]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), 
             callbacks = [ClearTrainingOutput()])

# Получаем оптимальные гиперпараметры
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Чтобы завершить это руководство, переобучите модель с оптимальными гиперпараметрами из поиска.

In [ ]:
# Строим модель с оптимальными гиперпараметрами и обучим ее на данных
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Каталог `my_dir/intro_to_kt` содержит подробные логи и контрольные точки для каждого испытания(конфигурации модели), запущенного во время поиска гиперпараметров. Если вы повторно запустите поиск гиперпараметров, Keras Tuner использует существующее состояние из этих журналов для возобновления поиска. Чтобы отключить это поведение, передайте дополнительный аргумент `overwrite = True` при создании экземпляра тюнера.

## Заключение

В этом руководстве вы узнали, как использовать Keras Tuner для настройки гиперпараметров модели. Чтобы узнать больше о Keras Tuner, ознакомьтесь с этими дополнительными ресурсами:

* [Keras Tuner в блоге TensorFlow](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html)
* [Веб-сайт Keras Tuner](https://keras-team.github.io/keras-tuner/)

Также посетите [Панель инструментов HParams](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams) в TensorBoard, чтобы интерактивно настроить гиперпараметры вашей модели.